In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

# Convert the 'date' column to datetime
data['date'] = pd.to_datetime(data['date'])

data = data.iloc[:10747]

data.isnull().sum()
data = data.fillna(0)  # Filling null values with zero

# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])

# Extract day, month, and year from the date column
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# Define the embedding dimensions
embedding_dim = 5  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Combine date_embeddings with the standardized features
input_data = torch.cat((torch.Tensor(data[columns_to_standardize].values), date_embeddings), dim=1)

# Split the data into training and validation sets
train_data, val_data = train_test_split(input_data.detach().numpy(), test_size=0.2, random_state=42)
train_data = torch.Tensor(train_data)
val_data = torch.Tensor(val_data)

# Define the autoencoder architecture
class TemporalAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TemporalAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        x = F.relu(self.encoder(x))
        x = self.decoder(x)
        return x

# Specify the input and hidden size for the autoencoder
input_size = input_data.shape[1]
hidden_size = 10  # You can adjust this size as needed

# Create the autoencoder model
autoencoder = TemporalAutoencoder(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Convert data to PyTorch DataLoader
train_dataset = TensorDataset(train_data, train_data)
val_dataset = TensorDataset(val_data, val_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Train the autoencoder
num_epochs = 1000  # You can adjust this number of epochs
for epoch in range(num_epochs):
    autoencoder.train()  # Set the model to training mode
    for data_batch, _ in train_loader:
        optimizer.zero_grad()
        reconstructions = autoencoder(data_batch)
        loss = criterion(reconstructions, data_batch)
        loss.backward()
        optimizer.step()

    # Evaluate on the validation set
    autoencoder.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        for val_batch, _ in val_loader:
            val_reconstructions = autoencoder(val_batch)
            val_loss += criterion(val_reconstructions, val_batch).item()

    # Print training and validation loss at each epoch
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')
    #print(f'Epoch [{epoch+100}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

# Evaluate the autoencoder on the training data
with torch.no_grad():
    autoencoder.eval()  # Set the model to evaluation mode
    reconstructions = autoencoder(train_data)
    mse = F.mse_loss(reconstructions, train_data)
    print(f'Mean Squared Error on Training Data: {mse.item():.4f}')

# Extract the learned temporal features from the encoder
with torch.no_grad():
    autoencoder.eval()
    encoded_data = autoencoder.encoder(input_data).detach().numpy()

# Now 'encoded_data' contains the learned temporal features
# You can use these features for further analysis or downstream tasks


Epoch [100/1000], Training Loss: 0.3947, Validation Loss: 0.2523
Epoch [200/1000], Training Loss: 0.2791, Validation Loss: 0.2480
Epoch [300/1000], Training Loss: 0.2611, Validation Loss: 0.2481
Epoch [400/1000], Training Loss: 0.2384, Validation Loss: 0.2478
Epoch [500/1000], Training Loss: 0.2951, Validation Loss: 0.2482
Epoch [600/1000], Training Loss: 0.2489, Validation Loss: 0.2478
Epoch [700/1000], Training Loss: 0.2534, Validation Loss: 0.2479
Epoch [800/1000], Training Loss: 0.2882, Validation Loss: 0.2478
Epoch [900/1000], Training Loss: 0.2630, Validation Loss: 0.2478
Epoch [1000/1000], Training Loss: 0.2441, Validation Loss: 0.2475
Mean Squared Error on Training Data: 0.2451


In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming 'encoded_data' is the result from the first model
x_train_encoded = torch.tensor(encoded_data, dtype=torch.float32)

# Load the y_train data
y_train = data["close"].to_numpy()

# Concatenate the encoded data with the target variable
x_train_combined = torch.cat((x_train_encoded, torch.tensor(y_train, dtype=torch.float32).view(-1, 1)), dim=1)

# Split the data into training and testing sets
x_train_combined, x_test_combined, _, y_test = train_test_split(
    x_train_combined, y_train, test_size=0.33, random_state=45
)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_combined_scaled = scaler.fit_transform(x_train_combined)
x_test_combined_scaled = scaler.transform(x_test_combined)

# Concatenate date embeddings with your feature vectors for training data
x_train_feature_tensors = torch.tensor(x_train_combined_scaled[:, :-1], dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_combined_scaled)]
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)

# Convert the combined data to PyTorch tensors
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_combined_scaled[:, :-1], dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_combined_scaled):]
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size, n_layers):
        super(LSTMModel, self).__init__()

        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, -1)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define hyperparameters
input_size = x_train_tensor.shape[1]
hidden_dim = 32
output_size = 1
n_layers = 4
learning_rate = 0.00015
batch_size = 64

# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_dim, output_size, n_layers).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Hidden Dim={hidden_dim}, Layers={n_layers}')

# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Print the predictions on the test set
with torch.no_grad():
    model.eval()
    test_predictions = model(x_test_tensor.view(x_test_tensor.size(0), 1, -1)).numpy()

# Print or use 'test_predictions' for further analysis
print(test_predictions)


C:\Users\pitr7\AppData\Local\Temp\ipykernel_8480\3393339538.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


NameError: name 'y_train_tensor' is not defined

1.- IMPORTING LIBRARIES: The script starts by importing the necessary libraries, including pandas, numpy, torch, and sklearn.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler


2.-DATA PREPROCESSING: The CSV file containing historical stock data is read using pandas.

In [ ]:
# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.head(1)


,date,open,high,low,close,volume,adjusted_close,change_percent,avg_vol_20d
0,1980-12-12,28.7392,28.8736,28.7392,28.7392,2093900,0.0994,NaN,NaN


In [ ]:
data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
#data.isnull().sum()


In [ ]:
correlation = data['close'].corr(data['adjusted_close'])
print(f"Correlation between 'close' and 'adjusted_close': {correlation:.2f}")


Correlation between 'close' and 'adjusted_close': 0.28


In [ ]:
correlation = data['volume'].corr(data['avg_vol_20d'])
print(f"Correlation between 'volume' and 'adjusted_close': {correlation:.2f}")


Correlation between 'volume' and 'adjusted_close': 0.05


In [ ]:
#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)


2.1.- Data Info: to get a summary of the dataset, including the count of non-null values.

In [ ]:
#data.info()


2.2.- describe() method to get summary statistics for the dataset, including the count, mean, and standard deviation.

In [ ]:
#data.describe()


2.3.- BOX PLOT is a graphical representation of the distribution of a dataset. It provides a concise summary of key statistical properties, such as the median, quartiles, and potential outliers within the data. Box plots are particularly useful for visualizing the spread and skewness of data and for identifying potential outliers.

In [ ]:
"""

import matplotlib.pyplot as plt
import pandas as pd



# Initialize an empty list to store valid column names
valid_columns = []

# Iterate through columns and check data type
for column_name in data.columns:
    if pd.api.types.is_numeric_dtype(data[column_name]):
        valid_columns.append(column_name)

# Create box plots for valid columns
for column_name in valid_columns:
    plt.figure()  # Create a new figure for each box plot
    plt.boxplot(data[column_name])
    plt.xlabel("X-axis label")
    plt.ylabel("Y-axis label")
    plt.title(f"Box Plot for {column_name}")

# Show all the plots
plt.show()
"""


'\n\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n\n\n# Initialize an empty list to store valid column names\nvalid_columns = []\n\n# Iterate through columns and check data type\nfor column_name in data.columns:\n    if pd.api.types.is_numeric_dtype(data[column_name]):\n        valid_columns.append(column_name)\n\n# Create box plots for valid columns\nfor column_name in valid_columns:\n    plt.figure()  # Create a new figure for each box plot\n    plt.boxplot(data[column_name])\n    plt.xlabel("X-axis label")\n    plt.ylabel("Y-axis label")\n    plt.title(f"Box Plot for {column_name}")\n\n# Show all the plots\nplt.show()\n'

2.4.- Heatmap to visualize CORRELATIONS between multiple variables in a dataset, making it easier to identify relationships and patterns in the data.

In [ ]:
"""

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 8))
heatmap = sns.heatmap(data.corr(), annot=True, fmt=".4f")  # Display two decimal places

# Get the labels from the DataFrame columns
labels = data.columns

# Set the labels on the x-axis (bottom) and y-axis (left) to be the variable names
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.show()

"""


'\n\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\nplt.figure(figsize=(8, 8))\nheatmap = sns.heatmap(data.corr(), annot=True, fmt=".4f")  # Display two decimal places\n\n# Get the labels from the DataFrame columns\nlabels = data.columns\n\n# Set the labels on the x-axis (bottom) and y-axis (left) to be the variable names\nplt.xticks(rotation=45, ha=\'right\')\nplt.yticks(rotation=0)\n\nplt.show()\n\n'

2.5.- PAIRPLOT: is a grid of scatterplots that shows the relationships between pairs of variables in a dataset.

In [ ]:
#sns.pairplot(data);


2.6.- Standardization (z-score normalization) transforms data to have a 
mean: 0 
standard deviation: 1 
making it comparable and suitable for machine learning; 
it helps with outlier handling and improves interpretability of model coefficients.

In [ ]:
# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])




2.7.-t-Distributed Stochastic Neighbor Embedding (t-SNE)

 is a dimensionality reduction technique that is widely used for data visualization and exploration. It helps uncover patterns and relationships within data by projecting it into a lower-dimensional space while preserving pairwise similarities among data points.

In [ ]:
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns

# Function to load and preprocess data
def load_and_preprocess_data():

    # Create the feature matrix 'x' and binarize the target 'y'
    x = data[["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]].to_numpy()

    y = np.where(data["close"] >= data["open"], 1, 0)

    return x, y

# Function to perform t-SNE and plot the results in 3D
def perform_tsne_and_plot_3d(x, y, feature_names):
    tsne = TSNE(n_iter=1000, n_components=3)  # Set n_components to 3 for 3D visualization
    embedding = tsne.fit_transform(x)

    # Create a 3D scatter plot with feature names as labels, legends, and title
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    palette = sns.color_palette("husl", 10)

    for i in range(10):
        ax.scatter(
            embedding[np.where(y == i), 0],
            embedding[np.where(y == i), 1],
            embedding[np.where(y == i), 2],
            label=feature_names[i],  # Use feature names as labels
            alpha=0.7, s=50, edgecolor='k', c=[palette[i]]
        )

    ax.legend(title="Feature Names", labels=feature_names)  # Set feature names as legend labels
    ax.set_title("Apple t-SNE 3D Embedding")
    plt.show()

# Main function
def main():
    # Load and preprocess data
    x, y = load_and_preprocess_data()

    # Define feature names corresponding to each class (0 to 9)
    feature_names = [
       "open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d",
    ]

    # Perform t-SNE and plot the results in 3D
    perform_tsne_and_plot_3d(x, y, feature_names)

if __name__ == "__main__":
    main()

"""


'\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom sklearn.manifold import TSNE\nimport seaborn as sns\n\n# Function to load and preprocess data\ndef load_and_preprocess_data():\n\n    # Create the feature matrix \'x\' and binarize the target \'y\'\n    x = data[["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]].to_numpy()\n\n    y = np.where(data["close"] >= data["open"], 1, 0)\n\n    return x, y\n\n# Function to perform t-SNE and plot the results in 3D\ndef perform_tsne_and_plot_3d(x, y, feature_names):\n    tsne = TSNE(n_iter=1000, n_components=3)  # Set n_components to 3 for 3D visualization\n    embedding = tsne.fit_transform(x)\n\n    # Create a 3D scatter plot with feature names as labels, legends, and title\n    fig = plt.figure(figsize=(12, 10))\n    ax = fig.add_subplot(111, projection=\'3d\')\n\n    palette = sns.color_palette("husl", 10)\n\n    for i in range(10):\n        ax.scatter(\n            embedding[n

2.8.-Hauptkomponentenanalyse (PCA) 

 is a dimensionality reduction technique. PCA helps in transforming high-dimensional data into a lower-dimensional form while preserving as much of the original data's variance as possible. It does this by identifying the principal components, which are linear combinations of the original features that capture the most significant variations in the data.

In [ ]:
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D  # Import for 3D plotting

# Load your data into the 'data' DataFrame
# Assuming you have a DataFrame named 'data'

# Create the feature matrix 'x' and binarize the target 'y'

x = data[["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]].to_numpy()

y = np.where(data["close"] >= data["open"], 1, 0)

# Define feature labels
feature_labels = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Perform PCA with 3 components (for 3D visualization)
pca = PCA(n_components=3)
embedding = pca.fit_transform(x)

# Create a 3D scatter plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Define a color palette with distinct colors for each class
palette = plt.cm.Paired(np.linspace(0, 1, 10))

for i in range(10):
    ax.scatter(embedding[np.where(y == i), 0], embedding[np.where(y == i), 1], embedding[np.where(y == i), 2],
               label=feature_labels[i], alpha=0.7, marker='o', s=50, edgecolor='k', c=[palette[i]])

ax.legend(loc='lower right')
ax.set_title("Apple PCA Embedding (3D)")

# Set axis labels
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.set_zlabel("Principal Component 3")

# Annotate points with feature labels (optional)
for i, label in enumerate(feature_labels):
    ax.text(embedding[:, 0][i], embedding[:, 1][i], embedding[:, 2][i], label, fontsize=8)

plt.show()
"""


'\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom sklearn.decomposition import PCA\nfrom mpl_toolkits.mplot3d import Axes3D  # Import for 3D plotting\n\n# Load your data into the \'data\' DataFrame\n# Assuming you have a DataFrame named \'data\'\n\n# Create the feature matrix \'x\' and binarize the target \'y\'\n\nx = data[["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]].to_numpy()\n\ny = np.where(data["close"] >= data["open"], 1, 0)\n\n# Define feature labels\nfeature_labels = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]\n\n# Perform PCA with 3 components (for 3D visualization)\npca = PCA(n_components=3)\nembedding = pca.fit_transform(x)\n\n# Create a 3D scatter plot\nfig = plt.figure(figsize=(12, 10))\nax = fig.add_subplot(111, projection=\'3d\')\n\n# Define a color palette with distinct colors for each class\npalette = plt.cm.Paired(np.linspace(0, 1, 10))\n\nfor i in range(10

In [ ]:
import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-09-12
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-09-12')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings
print(date_embeddings)


tensor([[-0.0446,  1.3953,  1.4894],
        [-0.7662,  1.3953,  1.4894],
        [ 0.7816,  1.3953,  1.4894],
        ...,
        [-0.0917,  1.9948, -1.9794],
        [-0.2234,  1.9948, -1.9794],
        [-0.0446,  1.9948, -1.9794]], grad_fn=<CatBackward0>)


In [ ]:
ö


NameError: name 'ö' is not defined

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)






# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out




# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')


# Training loop
num_epochs = 4000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            # Extract features from the hidden states
            hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        # You can now use 'hidden_states' as the feature representations of your sequences
        # The shape of 'hidden_states' will be (batch_size, sequence_length, hidden_dim)

# Print hidden_states after training
print(hidden_states)










C:\Users\pitr7\AppData\Local\Temp\ipykernel_9852\3844195813.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=0.00015, Sequence Length=75, Batch Size=64, Input Size=7, Date Embedding Dim=3, Hidden Dim=32,Layers=4
Epoch [100/4000], Loss: 17092.7461, Val Loss: 21942.2578
Epoch [200/4000], Loss: 14605.1865, Val Loss: 18084.5977
Epoch [300/4000], Loss: 8754.7090, Val Loss: 12628.4258
Epoch [400/4000], Loss: 12405.7861, Val Loss: 9659.0439
Epoch [500/4000], Loss: 11266.8896, Val Loss: 7365.3594
Epoch [600/4000], Loss: 4706.5659, Val Loss: 5510.0410
Epoch [700/4000], Loss: 2607.5483, Val Loss: 4017.1169
Epoch [800/4000], Loss: 5423.0728, Val Loss: 2837.3076
Epoch [900/4000], Loss: 2836.4011, Val Loss: 1936.3269
Epoch [1000/4000], Loss: 3388.9854, Val Loss: 1264.0769
Epoch [1100/4000], Loss: 1677.0354, Val Loss: 778.7593
Epoch [1200/4000], Loss: 1314.0870, Val Loss: 451.5746
Epoch [1300/4000], Loss: 33.3315, Val Loss: 239.9185
Epoch [1400/4000], Loss: 104.7675, Val Loss: 120.4431
Epoch [1500/4000], Loss: 234.0286, Val Loss: 57.6882
Epoch [1600/4000], Loss: 1.0510, Val L

Use the features extracted from the LSTM as input to a RANDOM FOREST model for a regression task

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Reshape the hidden states
reshaped_features = hidden_states.view(x_test_tensor.size(0), -1).numpy()

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(reshaped_features, y_test)

# Make predictions
rf_predictions = rf_model.predict(reshaped_features)

# Calculate RMSE (Root Mean Squared Error) or other evaluation metrics
rmse = mean_squared_error(y_test, rf_predictions, squared=False)
print(f'Random Forest RMSE: {rmse}')


Random Forest RMSE: 0.4796417605072094


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Reshape the hidden states
reshaped_features = hidden_states.view(x_test_tensor.size(0), -1).numpy()

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(reshaped_features, y_test)

# Make predictions
xgb_predictions = xgb_model.predict(reshaped_features)

# Calculate RMSE (Root Mean Squared Error) or other evaluation metrics
rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
print(f'XGBoost RMSE: {rmse}')


XGBoost RMSE: 0.6470004569099246


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Reshape the hidden states
reshaped_features = hidden_states.view(x_test_tensor.size(0), -1).numpy()

# List of models to iterate over
models = [xgb.XGBRegressor(n_estimators=100, random_state=42), RandomForestRegressor(n_estimators=100, random_state=42)]

for model in models:
    # Initialize and train the current model
    model.fit(reshaped_features, y_test)

    # Make predictions
    model_predictions = model.predict(reshaped_features)

    # Calculate RMSE
    rmse = mean_squared_error(y_test, model_predictions, squared=False)

    # Print RMSE for the current model
    print(f"Model: {type(model).__name__}, RMSE: {rmse}")


NameError: name 'hidden_states' is not defined

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out






# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 502
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')

# Training loop
num_epochs = 8000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Evaluate on testing set
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


C:\Users\pitr7\AppData\Local\Temp\ipykernel_5220\3494852810.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=0.00015, Sequence Length=75, Batch Size=64, Input Size=7, Date Embedding Dim=3, Hidden Dim=32,Layers=4
Epoch [100/8000], Loss: 17093.5117, Val Loss: 21943.0117
Epoch [200/8000], Loss: 13513.0918, Val Loss: 16716.0137
Epoch [300/8000], Loss: 8734.6221, Val Loss: 12607.3535
Epoch [400/8000], Loss: 12387.0977, Val Loss: 9643.1445
Epoch [500/8000], Loss: 11247.2510, Val Loss: 7352.6221
Epoch [600/8000], Loss: 4698.8184, Val Loss: 5499.7446
Epoch [700/8000], Loss: 2600.6853, Val Loss: 4008.8850
Epoch [800/8000], Loss: 5414.8291, Val Loss: 2830.9409
Epoch [900/8000], Loss: 2829.0701, Val Loss: 1931.5015
Epoch [1000/8000], Loss: 3382.1975, Val Loss: 1260.4774
Epoch [1100/8000], Loss: 1672.0790, Val Loss: 776.2928
Epoch [1200/8000], Loss: 1310.7181, Val Loss: 449.7979
Epoch [1300/8000], Loss: 32.9099, Val Loss: 238.8957
Epoch [1400/8000], Loss: 104.1758, Val Loss: 119.7787
Epoch [1500/8000], Loss: 233.3927, Val Loss: 57.3146
Epoch [1600/8000], Loss: 1.1345, Val L

KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out






# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')

# Training loop
num_epochs = 8000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Evaluate on testing set
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


: 

 LSTM model with hyperparameter optimization using Optuna

 Optuna is generally more computationally efficient than traditional grid search and evolutionary parameter optimization methods.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)

# Convert the combined data to PyTorch tensors
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)

# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Define the objective function for hyperparameter optimization
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    n_layers = trial.suggest_int('n_layers', 2, 6)
    sequence_length = trial.suggest_int('sequence_length', 1, 10)

    # Create an instance of the LSTM model with sampled hyperparameters
    model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

    # Define loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    num_epochs = 1000  # You can adjust the number of epochs

    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_x)

            # Calculate loss
            loss = loss_function(outputs, batch_y.view(-1, 1))

            # Backpropagation
            loss.backward()
            optimizer.step()

    # Evaluate on testing set
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_test_tensor)
        val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))

    return val_loss.item()

# Create a study for hyperparameter optimization
study = optuna.create_study(direction='minimize')  # Minimize validation loss
study.optimize(objective, n_trials=100)  # You can adjust the number of trials

# Get the best hyperparameters from the study
best_params = study.best_params

# Print the best hyperparameters
print("Best Hyperparameters:")
print("Learning Rate:", best_params['learning_rate'])
print("Batch Size:", best_params['batch_size'])
print("Hidden Dim:", best_params['hidden_dim'])
print("Number of Layers:", best_params['n_layers'])
print("Sequence Length:", best_params['sequence_length'])


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-10-09 11:53:39,130] A new study created in memory with name: no-name-0003f577-e84d-47d5-b2ab-8207026650fe
C:\Users\pitr7\AppData\Local\Temp\ipykernel_4832\1271520721.py:99: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
[I 2023-10-09 12:52:28,808] Trial 0 finished with value: 1587.290283203125 and parameters: {'learning_rate': 0.00011981449738351975, 'batch_size': 32, 'hidden_dim': 38, 'n_layers': 3, 'sequence_length': 1}. Best is trial 0 with value: 1587.290283203125.
[I 2023-10-09 15:32:13,099

KeyboardInterrupt: 

Evolutionary hyperparameter optimization:

is a technique that uses evolutionary algorithms to search for optimal hyperparameters for machine learning models. One common evolutionary algorithm used for this purpose is Genetic Algorithm (GA).

Certainly! Genetic Algorithms (GAs) are a type of optimization algorithm inspired by the process of natural selection and genetics. They are used to find approximate solutions to complex optimization and search problems. GAs work by evolving a population of candidate solutions over multiple generations to improve their fitness or objective function value. Here are the key components and steps of a typical GA:

Initialization:

Create an initial population of candidate solutions (often random).
Each candidate solution represents a possible solution to the problem, typically encoded as a set of parameters or variables.
Evaluation:

Calculate the fitness or objective function value for each candidate solution in the population.
The fitness function quantifies how good each solution is with respect to the optimization problem. It guides the search for better solutions.
Selection:

Select a subset of candidate solutions (parents) from the current population to create the next generation.
Solutions with higher fitness values are more likely to be selected.
Common selection methods include roulette wheel selection, tournament selection, and rank-based selection.
Crossover (Recombination):

Combine pairs of selected parent solutions to produce offspring solutions.
Crossover typically involves swapping or recombining parts of parent solutions to create one or more offspring solutions.
The goal is to introduce diversity and combine the good features of different solutions.
Mutation:

Apply small random changes to some of the offspring solutions.
Mutation helps to introduce new genetic material into the population and prevents premature convergence.
Replacement:

Create the next generation by combining the original population, offspring, and potentially some survivors from the previous generation.
The new population replaces the old population, and the cycle repeats.
Termination:

Determine when to stop the GA. This can be based on the number of generations, convergence criteria, or other stopping conditions.
Optionally, the best solution found during the optimization can be returned as the final result.
Convergence:

GAs tend to improve the quality of solutions over generations, and they converge toward optimal or near-optimal solutions. 

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from deap import base, creator, tools, algorithms
import torch

# Define a function to evaluate the fitness of an individual
def evaluate(individual):
    # Extract hyperparameters from the individual
    learning_rate, sequence_length, batch_size, hidden_dim, n_layers = individual

    # Create and train the LSTM model (You need to define these functions)
    model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop (You need to define train_loader, loss_function, num_epochs, x_test_tensor, y_test_tensor)
    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = loss_function(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_test_tensor)
        val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))

    # Return the validation loss as the fitness value
    return val_loss.item(),

# Define the optimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

# Register the evaluate function in the toolbox
toolbox.register("evaluate", evaluate)

# Define the search space for hyperparameters
toolbox.register("attr_float", np.random.uniform, 1e-5, 1e-3)  # Learning rate
toolbox.register("attr_int_seq_length", np.random.randint, 1, 200)         # Sequence length
toolbox.register("attr_int_batch_size", np.random.randint, 16, 128)        # Batch size
toolbox.register("attr_int_hidden_dim", np.random.randint, 16, 128)        # Hidden dimension
toolbox.register("attr_int_n_layers", np.random.randint, 1, 5)           # Number of layers

# Define the individual and population size
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float, toolbox.attr_int_seq_length,
                                                                     toolbox.attr_int_batch_size, toolbox.attr_int_hidden_dim,
                                                                     toolbox.attr_int_n_layers), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the genetic operators (crossover and mutation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)

# Define the selection operator
toolbox.register("select", tools.selTournament, tournsize=3)

# Define the statistics to track during optimization
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min)

# Create the initial population
population = toolbox.population(n=10)

# Create the Hall of Fame to store the best individuals
hof = tools.HallOfFame(1)

# Set the algorithm parameters
ngen = 10   # Number of generations
cxpb = 0.7  # Crossover probability
mutpb = 0.2 # Mutation probability

# Run the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats, halloffame=hof, verbose=True)

# Get the best individual (optimized hyperparameters)
best_individual = hof[0]
best_learning_rate, best_sequence_length, best_batch_size, best_hidden_dim, best_n_layers = best_individual

print("Best Hyperparameters:")
print(f"Learning Rate: {best_learning_rate}")
print(f"Sequence Length: {best_sequence_length}")
print(f"Batch Size: {best_batch_size}")
print(f"Hidden Dimension: {best_hidden_dim}")
print(f"Number of Layers: {best_n_layers}")


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 

Bayessche Hyperparmeter optimierung

Ba

In [ ]:
import optuna

# Define the objective function for Optuna hyperparameter optimization
def objective(trial):
    # Define a search space for hyperparameters

    input_size = 7
    date_embedding_dim = 3
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    sequence_length = trial.suggest_int('sequence_length', 1, 100)
    batch_size = trial.suggest_int('batch_size',16,128)
    n_layers = trial.suggest_int('n_layers', 1, 4)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)



    # Create an instance of the LSTM model with the suggested hyperparameters
    model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

    # Define loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    num_epochs = 4000

    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_x)

            # Calculate loss
            loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

            # Backpropagation
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 100 == 0:
            # Evaluate on testing set
            model.eval()
            with torch.no_grad():
                val_outputs = model(x_test_tensor)
                val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
            print(f'Trial {trial.number}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    # Calculate the final validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_test_tensor)
        val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape

    print(f'Trial {trial.number}, Final Val Loss: {val_loss.item():.4f}')

    return val_loss.item()

# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')  # We want to minimize the validation loss
study.optimize(objective, n_trials=100)  # You can adjust the number of trials as needed

# Retrieve the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


[I 2023-10-05 23:18:08,179] A new study created in memory with name: no-name-1cd1821b-9136-450d-a09d-cebe054b0453


Trial 0, Epoch [100/4000], Loss: 0.4427, Val Loss: 286.9637
Trial 0, Epoch [200/4000], Loss: 3.0529, Val Loss: 2.4061
Trial 0, Epoch [300/4000], Loss: 3.6636, Val Loss: 1.9211
Trial 0, Epoch [400/4000], Loss: 0.7943, Val Loss: 1.9691
Trial 0, Epoch [500/4000], Loss: 1.9298, Val Loss: 2.7451
Trial 0, Epoch [600/4000], Loss: 0.8580, Val Loss: 1.8333
Trial 0, Epoch [700/4000], Loss: 3.1275, Val Loss: 1.8677
Trial 0, Epoch [800/4000], Loss: 0.8538, Val Loss: 1.8187
Trial 0, Epoch [900/4000], Loss: 1.1385, Val Loss: 1.6672
Trial 0, Epoch [1000/4000], Loss: 1.1785, Val Loss: 1.7389
Trial 0, Epoch [1100/4000], Loss: 0.8306, Val Loss: 1.7341
Trial 0, Epoch [1200/4000], Loss: 0.7424, Val Loss: 1.7269
Trial 0, Epoch [1300/4000], Loss: 0.5770, Val Loss: 2.0358
Trial 0, Epoch [1400/4000], Loss: 0.5814, Val Loss: 1.6488
Trial 0, Epoch [1500/4000], Loss: 0.7378, Val Loss: 1.7816
Trial 0, Epoch [1600/4000], Loss: 1.2634, Val Loss: 1.9273
Trial 0, Epoch [1700/4000], Loss: 0.5632, Val Loss: 2.0644
Tria

[I 2023-10-06 02:01:39,550] Trial 0 finished with value: 2.543423652648926 and parameters: {'hidden_dim': 76, 'sequence_length': 91, 'batch_size': 49, 'n_layers': 2, 'learning_rate': 0.001056769059573483}. Best is trial 0 with value: 2.543423652648926.


Trial 1, Epoch [100/4000], Loss: 3.2198, Val Loss: 3.0797
Trial 1, Epoch [200/4000], Loss: 2.1986, Val Loss: 4.0407
Trial 1, Epoch [300/4000], Loss: 1.4162, Val Loss: 1.8597
Trial 1, Epoch [400/4000], Loss: 1.3368, Val Loss: 2.0725
Trial 1, Epoch [500/4000], Loss: 2.7763, Val Loss: 2.0355
Trial 1, Epoch [600/4000], Loss: 1.0290, Val Loss: 2.0713
Trial 1, Epoch [700/4000], Loss: 0.9509, Val Loss: 3.6938
Trial 1, Epoch [800/4000], Loss: 1.6594, Val Loss: 2.5067
Trial 1, Epoch [900/4000], Loss: 0.8237, Val Loss: 1.9579
Trial 1, Epoch [1000/4000], Loss: 0.9927, Val Loss: 2.6719
Trial 1, Epoch [1100/4000], Loss: 0.4332, Val Loss: 1.8620
Trial 1, Epoch [1200/4000], Loss: 0.8185, Val Loss: 2.3024
Trial 1, Epoch [1300/4000], Loss: 1.3519, Val Loss: 2.3728
Trial 1, Epoch [1400/4000], Loss: 0.9412, Val Loss: 2.1581
Trial 1, Epoch [1500/4000], Loss: 0.6877, Val Loss: 2.5076
Trial 1, Epoch [1600/4000], Loss: 0.8318, Val Loss: 2.3031
Trial 1, Epoch [1700/4000], Loss: 0.5837, Val Loss: 2.5397
Trial 

[I 2023-10-06 05:57:05,242] Trial 1 finished with value: 4.636506080627441 and parameters: {'hidden_dim': 84, 'sequence_length': 9, 'batch_size': 72, 'n_layers': 4, 'learning_rate': 0.0024304404022470315}. Best is trial 0 with value: 2.543423652648926.


...

Trial 7, Epoch [4000/4000], Loss: 0.3811, Val Loss: 82.6979
Trial 7, Final Val Loss: 82.6979
Trial 8, Epoch [100/4000], Loss: 2.5486, Val Loss: 2.1391
Trial 8, Epoch [200/4000], Loss: 1.1248, Val Loss: 2.8298
Trial 8, Epoch [300/4000], Loss: 1.3750, Val Loss: 2.5605
Trial 8, Epoch [400/4000], Loss: 0.7264, Val Loss: 3.0075
Trial 8, Epoch [500/4000], Loss: 0.7262, Val Loss: 2.7667
Trial 8, Epoch [600/4000], Loss: 0.8390, Val Loss: 4.5630
Trial 8, Epoch [700/4000], Loss: 1.2667, Val Loss: 3.5549
Trial 8, Epoch [800/4000], Loss: 1.2346, Val Loss: 2.6030
Trial 8, Epoch [900/4000], Loss: 0.8426, Val Loss: 2.4265
Trial 8, Epoch [1000/4000], Loss: 1.1194, Val Loss: 1.9970
Trial 8, Epoch [1100/4000], Loss: 2.9854, Val Loss: 2.0844
Trial 8, Epoch [1200/4000], Loss: 0.7369, Val Loss: 2.1456
Trial 8, Epoch [1300/4000], Loss: 1.1705, Val Loss: 2.5637
Trial 8, Epoch [1400/4000], Loss: 0.8405, Val Loss: 4.4944
Trial 8, Epoch [1500/4000], Loss: 0.9368, Val Loss: 6.2019
Trial 8, Epoch [1600/4000], Los

[I 2023-10-07 14:04:39,930] Trial 8 finished with value: 2.946018934249878 and parameters: {'hidden_dim': 75, 'sequence_length': 3, 'batch_size': 25, 'n_layers': 1, 'learning_rate': 0.009346372189458564}. Best is trial 0 with value: 2.543423652648926.


Trial 9, Epoch [100/4000], Loss: 1.6712, Val Loss: 5.1696
Trial 9, Epoch [200/4000], Loss: 1.4234, Val Loss: 2.7799
Trial 9, Epoch [300/4000], Loss: 1.6638, Val Loss: 1.6209
Trial 9, Epoch [400/4000], Loss: 1.4100, Val Loss: 1.7805
Trial 9, Epoch [500/4000], Loss: 0.7871, Val Loss: 2.0007
Trial 9, Epoch [600/4000], Loss: 1.0089, Val Loss: 2.1835
Trial 9, Epoch [700/4000], Loss: 0.7824, Val Loss: 1.7645
Trial 9, Epoch [800/4000], Loss: 0.7452, Val Loss: 1.6886
Trial 9, Epoch [900/4000], Loss: 0.8755, Val Loss: 2.0400
Trial 9, Epoch [1000/4000], Loss: 1.2282, Val Loss: 1.8729
Trial 9, Epoch [1100/4000], Loss: 1.2586, Val Loss: 2.3768
Trial 9, Epoch [1200/4000], Loss: 0.3321, Val Loss: 1.8090
Trial 9, Epoch [1300/4000], Loss: 0.2516, Val Loss: 1.6202
Trial 9, Epoch [1400/4000], Loss: 1.4296, Val Loss: 2.0018
Trial 9, Epoch [1500/4000], Loss: 0.3880, Val Loss: 1.9152
Trial 9, Epoch [1600/4000], Loss: 1.0906, Val Loss: 2.1916
Trial 9, Epoch [1700/4000], Loss: 0.4642, Val Loss: 2.0720
Trial 

[W 2023-10-07 17:25:04,757] Trial 9 failed with parameters: {'hidden_dim': 126, 'sequence_length': 69, 'batch_size': 77, 'n_layers': 3, 'learning_rate': 0.0013957939123843718} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\pitr7\AppData\Local\Temp\ipykernel_5220\3950734113.py", line 33, in objective
    outputs = model(batch_x)
  File "c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "C:\Users\pitr7\AppData\Local\Temp\ipykernel_5220\3494852810.py", line 61, in forward
    out, (hidden, cell) = self.lstm(x, (h0, c0))
  File "c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)

KeyboardInterrupt: 

In [ ]:
import optuna

# Define the objective function for Optuna hyperparameter optimization
def objective(trial):
    # Define a search space for hyperparameters
    input_size = trial.suggest_int('input_size', 5, 20)
    date_embedding_dim = trial.suggest_int('date_embedding_dim', 2, 10)
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    n_layers = trial.suggest_int('n_layers', 1, 4)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)

    # Create an instance of the LSTM model with the suggested hyperparameters
    model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

    # Define loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    num_epochs = 4000

    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_x)

            # Calculate loss
            loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

            # Backpropagation
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 100 == 0:
            # Evaluate on testing set
            model.eval()
            with torch.no_grad():
                val_outputs = model(x_test_tensor)
                val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
            print(f'Trial {trial.number}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    # Calculate the final validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_test_tensor)
        val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape

    print(f'Trial {trial.number}, Final Val Loss: {val_loss.item():.4f}')

    return val_loss.item()

# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')  # We want to minimize the validation loss
study.optimize(objective, n_trials=100)  # You can adjust the number of trials as needed

# Retrieve the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


ModuleNotFoundError: No module named 'optuna'

In [ ]:
ö


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out






# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 3
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')

# Training loop
num_epochs = 8000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Evaluate on testing set
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9648\37430299.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=0.00015, Sequence Length=75, Batch Size=64, Input Size=7, Date Embedding Dim=3, Hidden Dim=32,Layers=3
Epoch [100/8000], Loss: 21872.4414, Val Loss: 21901.0703
Epoch [200/8000], Loss: 6361.3525, Val Loss: 16445.4980
Epoch [300/8000], Loss: 1646.8357, Val Loss: 12494.5293
Epoch [400/8000], Loss: 4116.4194, Val Loss: 9560.4355
Epoch [500/8000], Loss: 10061.7539, Val Loss: 7283.7065
Epoch [600/8000], Loss: 4822.4604, Val Loss: 5441.8330
Epoch [700/8000], Loss: 4601.1895, Val Loss: 3959.5132
Epoch [800/8000], Loss: 2730.0044, Val Loss: 2792.3157
Epoch [900/8000], Loss: 2838.2622, Val Loss: 1901.4434
Epoch [1000/8000], Loss: 3543.3816, Val Loss: 1236.9226
Epoch [1100/8000], Loss: 835.0153, Val Loss: 761.1047
Epoch [1200/8000], Loss: 134.0529, Val Loss: 439.4286
Epoch [1300/8000], Loss: 1.0507, Val Loss: 233.5591
Epoch [1400/8000], Loss: 0.8516, Val Loss: 116.6809
Epoch [1500/8000], Loss: 28.6091, Val Loss: 55.5436
Epoch [1600/8000], Loss: 0.3527, Val Loss: 25.

In [ ]:
import optuna

# Define the objective function for Optuna hyperparameter optimization
def objective(trial):
    # Define a search space for hyperparameters
    input_size = trial.suggest_int('input_size', 5, 20)
    date_embedding_dim = trial.suggest_int('date_embedding_dim', 2, 10)
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    n_layers = trial.suggest_int('n_layers', 1, 4)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)

    # Create an instance of the LSTM model with the suggested hyperparameters
    model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

    # Define loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    num_epochs = 4000

    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_x)

            # Calculate loss
            loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

            # Backpropagation
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 100 == 0:
            # Evaluate on testing set
            model.eval()
            with torch.no_grad():
                val_outputs = model(x_test_tensor)
                val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
            print(f'Trial {trial.number}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    # Calculate the final validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_test_tensor)
        val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape

    print(f'Trial {trial.number}, Final Val Loss: {val_loss.item():.4f}')

    return val_loss.item()

# Create an Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')  # We want to minimize the validation loss
study.optimize(objective, n_trials=100)  # You can adjust the number of trials as needed

# Retrieve the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


ModuleNotFoundError: No module named 'optuna'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out






# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 3
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')

# Training loop
num_epochs = 8000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Evaluate on testing set
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9648\176290703.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=1e-05, Sequence Length=75, Batch Size=64, Input Size=7, Date Embedding Dim=3, Hidden Dim=32,Layers=3
Epoch [100/8000], Loss: 28950.9043, Val Loss: 29036.4414
Epoch [200/8000], Loss: 15068.4043, Val Loss: 28256.5996
Epoch [300/8000], Loss: 7812.2471, Val Loss: 27665.1289
Epoch [400/8000], Loss: 16380.8115, Val Loss: 27101.8984
Epoch [500/8000], Loss: 32320.1777, Val Loss: 26556.0469
Epoch [600/8000], Loss: 24506.4375, Val Loss: 26026.7324
Epoch [700/8000], Loss: 24143.5723, Val Loss: 25513.6387
Epoch [800/8000], Loss: 35430.0117, Val Loss: 25016.9082
Epoch [900/8000], Loss: 25550.9727, Val Loss: 24536.6582
Epoch [1000/8000], Loss: 42544.4922, Val Loss: 24072.6230
Epoch [1100/8000], Loss: 27106.0723, Val Loss: 23625.0059
Epoch [1200/8000], Loss: 19985.8086, Val Loss: 23193.7754
Epoch [1300/8000], Loss: 9424.4775, Val Loss: 22778.9414
Epoch [1400/8000], Loss: 12648.0938, Val Loss: 22380.5137
Epoch [1500/8000], Loss: 18165.0078, Val Loss: 21998.2930
Epoch [16